Notebook for the one-hot autoencoder

In [1]:
from communications import *
from model import *
import numpy as np
import matplotlib.pyplot as plt
import os.path

%load_ext autoreload
%autoreload 2

In [2]:
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
#X_train,y_train=np.eye(256),np.eye(256)
X_train,y_train=createBitVectors(),createBitVectors()
#X_train,y_train=createBitVectors(),2.0*createPolarCodewords()-1


In [4]:
SNR=3
add_name='_3_mish2'

In [ ]:
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder(5*256,5*256,AWGN,SNR,use_BN=False,use_LN=False)

In [ ]:
boundaries = [2**16,2**17]
values = [1e-4,1e-5,1e-6]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder2(5*256,5*256,AWGN,SNR,use_BN=True,use_LN=False,lr=1e-4,encoder_activation='tanh',hidden_activation='relu',optim=Adam,lookahead=False,gradient_centralization=False)

In [ ]:
autoencoderAWGN,encoderAWGN,decoderAWGN=OHAutoencoder(5*256,5*256,multi_AWGN,use_BN=False,use_LN=False)

In [ ]:
def schedule(epoch,lr):
    if epoch<2**13:
        return 1e-3
    if epoch<3*2**16:
        return 1e-5
    return 1e-6
    

scheduler=tf.keras.callbacks.LearningRateScheduler(schedule)


In [ ]:
from tensorflow.keras.models import load_model
#autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('normal','normal',[64,128],[128,64,32],AWGN,SNR,use_BN=True,use_LN=False,lr=1e-3,encoder_activation='tanh',hidden_activation=mish,optim=RectifiedAdam,lookahead=True,gradient_centralization=True)
encoderAWGN=load_model('AWGNEncoder_3_relu.h5',custom_objects={'AWGN':AWGN,'normLayer':normLayer})
#autoencoderAWGN,encoderAWGN,decoderAWGN=recompileAutoencoder('normal','normal',encoderAWGN,decoderAWGN,AWGN,noise_param=SNR,optim=Adam,lookahead=True,gradient_centralization=True,lr=1e-5)

In [5]:
autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('normal','normal',[64,128],[128,64],AWGN,SNR,use_BN=True,use_LN=False,use_WN=False,lr=1e-3,encoder_activation=mish,hidden_activation=mish,optim=RectifiedAdam,lookahead=True,gradient_centralization=True,layer_normalization=True,force_normalization=False,compile_autoencoder=True,compile_encoder=False,compile_decoder=False)

In [ ]:
boundaries = [2**16,2**17]
values = [1e-4,1e-5,1e-6]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('onehot','normal',1280,[1280],AWGN,SNR,use_BN=True,use_LN=False,lr=1e-3,encoder_activation='tanh',hidden_activation=mish,optim=RectifiedAdam,lookahead=True,gradient_centralization=True)

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint('AWGNAutoencoderCheckpoint'+add_name+'.h5',monitor='loss',save_best_only=True)
history=encoderAWGN.fit(x=X_train,y=y_train,batch_size=256,epochs=2**15,callbacks=[checkpoint,scheduler],verbose=2)
save_obj(history.history,'history'+add_name)
encoderAWGN.save('AWGNEncoder'+add_name+'.h5')

In [ ]:
print(np.std(encoderAWGN.predict(X_train)))

In [ ]:
print(y_train)

In [ ]:
from tensorflow.keras.models import load_model
#autoencoderAWGN,encoderAWGN,decoderAWGN=Autoencoder('normal','normal',[64,128],[128,64,32],AWGN,SNR,use_BN=True,use_LN=False,lr=1e-3,encoder_activation='tanh',hidden_activation=mish,optim=RectifiedAdam,lookahead=True,gradient_centralization=True)
encoderAWGN=load_model('AWGNEncoder_3_polarencoder.h5',custom_objects={'AWGN':AWGN,'polar_metric':polar_metric})
#autoencoderAWGN,encoderAWGN,decoderAWGN=recompileAutoencoder('normal','normal',encoderAWGN,decoderAWGN,AWGN,noise_param=SNR,optim=Adam,lookahead=True,gradient_centralization=True,lr=1e-5)

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint('AWGNAutoencoderCheckpoint'+add_name+'.h5',monitor='loss',save_best_only=True)
history=autoencoderAWGN.fit(x=X_train,y=y_train,batch_size=256,epochs=2**16,callbacks=[checkpoint],verbose=2)
save_obj(history.history,'history'+add_name)
autoencoderAWGN.save('AWGNAutoencoder'+add_name+'.h5')
encoderAWGN.save('AWGNEncoder'+add_name+'.h5')
decoderAWGN.save('AWGNDecoder'+add_name+'.h5')

Epoch 1/65536
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
1/1 - 0s - loss: 0.8686 - ber_metric: 0.5039 - acc: 0.0938
Epoch 2/65536
1/1 - 0s - loss: 0.8636 - ber_metric: 0.4922 - acc: 0.1211
Epoch 3/65536
1/1 - 0s - loss: 0.8512 - ber_metric: 0.4878 - acc: 0.1133
Epoch 4/65536
1/1 - 0s - loss: 0.8696 - ber_metric: 0.4868 - acc: 0.1094
Epoch 5/65536
1/1 - 0s - loss: 0.8660 - ber_metric: 0.4829 - acc: 0.1250
Epoch 6/65536
1/1 - 0s - loss: 0.8633 - ber_metric: 0.4917 - acc: 0.1328
Epoch 7/65536
1/1 - 0s - loss: 0.8581 - ber_metric: 0.4912 - acc: 0.1406
Epoch 8/65536
1/1 - 0s - loss: 0.8592 - ber_metric: 0.4858 - acc: 0.1211
Epoch 9/65536
1/1 - 0s - loss: 0.8483 - ber_metric: 0.4756 - acc: 0.1172
Epoch 10/65536
1/1 - 0s - loss: 0.8429 - ber_metric: 0.4756 - acc: 0.0898
Epoch 11/65536
1/1 - 0s - loss: 0.8295 - ber_metric: 0.4678 - acc: 0.1367
Epoch 12/65536
1/1 - 0s - loss: 0.8454 - ber_metric: 0.4692 - acc: 0.1133
Epoch 13/65536
1/1 - 0s - loss: 0.8453 -

Epoch 107/65536
1/1 - 0s - loss: 0.4307 - ber_metric: 0.1895 - acc: 0.1992
Epoch 108/65536
1/1 - 0s - loss: 0.4181 - ber_metric: 0.1768 - acc: 0.1875
Epoch 109/65536
1/1 - 0s - loss: 0.4348 - ber_metric: 0.1919 - acc: 0.1992
Epoch 110/65536
1/1 - 0s - loss: 0.4286 - ber_metric: 0.1782 - acc: 0.2344
Epoch 111/65536
1/1 - 0s - loss: 0.4156 - ber_metric: 0.1831 - acc: 0.2188
Epoch 112/65536
1/1 - 0s - loss: 0.4085 - ber_metric: 0.1709 - acc: 0.2344
Epoch 113/65536
1/1 - 0s - loss: 0.4135 - ber_metric: 0.1763 - acc: 0.2344
Epoch 114/65536
1/1 - 0s - loss: 0.4120 - ber_metric: 0.1724 - acc: 0.2422
Epoch 115/65536
1/1 - 0s - loss: 0.4092 - ber_metric: 0.1758 - acc: 0.2031
Epoch 116/65536
1/1 - 0s - loss: 0.4185 - ber_metric: 0.1782 - acc: 0.2188
Epoch 117/65536
1/1 - 0s - loss: 0.4096 - ber_metric: 0.1699 - acc: 0.2070
Epoch 118/65536
1/1 - 0s - loss: 0.4080 - ber_metric: 0.1738 - acc: 0.2031
Epoch 119/65536
1/1 - 0s - loss: 0.4086 - ber_metric: 0.1792 - acc: 0.2070
Epoch 120/65536
1/1 - 0s 

Epoch 217/65536
1/1 - 0s - loss: 0.2901 - ber_metric: 0.1045 - acc: 0.2539
Epoch 218/65536
1/1 - 0s - loss: 0.2927 - ber_metric: 0.1138 - acc: 0.2578
Epoch 219/65536
1/1 - 0s - loss: 0.2916 - ber_metric: 0.1064 - acc: 0.2305
Epoch 220/65536
1/1 - 0s - loss: 0.2864 - ber_metric: 0.1040 - acc: 0.2305
Epoch 221/65536
1/1 - 0s - loss: 0.2839 - ber_metric: 0.1001 - acc: 0.2188
Epoch 222/65536
1/1 - 0s - loss: 0.2952 - ber_metric: 0.1089 - acc: 0.2422
Epoch 223/65536
1/1 - 0s - loss: 0.2853 - ber_metric: 0.1040 - acc: 0.2891
Epoch 224/65536
1/1 - 0s - loss: 0.2835 - ber_metric: 0.1025 - acc: 0.2578
Epoch 225/65536
1/1 - 0s - loss: 0.2888 - ber_metric: 0.1060 - acc: 0.2930
Epoch 226/65536
1/1 - 0s - loss: 0.2864 - ber_metric: 0.1060 - acc: 0.2656
Epoch 227/65536
1/1 - 0s - loss: 0.2812 - ber_metric: 0.1045 - acc: 0.2930
Epoch 228/65536
1/1 - 0s - loss: 0.2890 - ber_metric: 0.1050 - acc: 0.2812
Epoch 229/65536
1/1 - 0s - loss: 0.2841 - ber_metric: 0.1030 - acc: 0.2422
Epoch 230/65536
1/1 - 0s 

Epoch 327/65536
1/1 - 0s - loss: 0.2347 - ber_metric: 0.0815 - acc: 0.2734
Epoch 328/65536
1/1 - 0s - loss: 0.2302 - ber_metric: 0.0742 - acc: 0.2578
Epoch 329/65536
1/1 - 0s - loss: 0.2188 - ber_metric: 0.0708 - acc: 0.2812
Epoch 330/65536
1/1 - 0s - loss: 0.2246 - ber_metric: 0.0728 - acc: 0.3047
Epoch 331/65536
1/1 - 0s - loss: 0.2188 - ber_metric: 0.0708 - acc: 0.2852
Epoch 332/65536
1/1 - 0s - loss: 0.2076 - ber_metric: 0.0664 - acc: 0.3242
Epoch 333/65536
1/1 - 0s - loss: 0.2231 - ber_metric: 0.0747 - acc: 0.2734
Epoch 334/65536
1/1 - 0s - loss: 0.2217 - ber_metric: 0.0723 - acc: 0.2461
Epoch 335/65536
1/1 - 0s - loss: 0.2296 - ber_metric: 0.0840 - acc: 0.2891
Epoch 336/65536
1/1 - 0s - loss: 0.2273 - ber_metric: 0.0820 - acc: 0.2969
Epoch 337/65536
1/1 - 0s - loss: 0.2250 - ber_metric: 0.0757 - acc: 0.2891
Epoch 338/65536
1/1 - 0s - loss: 0.2172 - ber_metric: 0.0728 - acc: 0.2930
Epoch 339/65536
1/1 - 0s - loss: 0.2164 - ber_metric: 0.0723 - acc: 0.2891
Epoch 340/65536
1/1 - 0s 

Epoch 437/65536
1/1 - 0s - loss: 0.1723 - ber_metric: 0.0527 - acc: 0.3555
Epoch 438/65536
1/1 - 0s - loss: 0.1677 - ber_metric: 0.0566 - acc: 0.2969
Epoch 439/65536
1/1 - 0s - loss: 0.1675 - ber_metric: 0.0493 - acc: 0.2812
Epoch 440/65536
1/1 - 0s - loss: 0.1744 - ber_metric: 0.0532 - acc: 0.2461
Epoch 441/65536
1/1 - 0s - loss: 0.1743 - ber_metric: 0.0513 - acc: 0.2930
Epoch 442/65536
1/1 - 0s - loss: 0.1739 - ber_metric: 0.0522 - acc: 0.2695
Epoch 443/65536
1/1 - 0s - loss: 0.1644 - ber_metric: 0.0459 - acc: 0.2891
Epoch 444/65536
1/1 - 0s - loss: 0.1682 - ber_metric: 0.0454 - acc: 0.3125
Epoch 445/65536
1/1 - 0s - loss: 0.1657 - ber_metric: 0.0469 - acc: 0.2539
Epoch 446/65536
1/1 - 0s - loss: 0.1727 - ber_metric: 0.0503 - acc: 0.2812
Epoch 447/65536
1/1 - 0s - loss: 0.1588 - ber_metric: 0.0400 - acc: 0.2891
Epoch 448/65536
1/1 - 0s - loss: 0.1677 - ber_metric: 0.0508 - acc: 0.2891
Epoch 449/65536
1/1 - 0s - loss: 0.1645 - ber_metric: 0.0483 - acc: 0.3047
Epoch 450/65536
1/1 - 0s 

...

Epoch 21362/65536
1/1 - 0s - loss: 0.0066 - ber_metric: 0.0034 - acc: 0.5195
Epoch 21363/65536
1/1 - 0s - loss: 0.0072 - ber_metric: 0.0024 - acc: 0.5000
Epoch 21364/65536
1/1 - 0s - loss: 0.0026 - ber_metric: 9.7656e-04 - acc: 0.5430
Epoch 21365/65536
1/1 - 0s - loss: 0.0031 - ber_metric: 4.8828e-04 - acc: 0.4766
Epoch 21366/65536
1/1 - 0s - loss: 0.0150 - ber_metric: 0.0034 - acc: 0.5352
Epoch 21367/65536
1/1 - 0s - loss: 0.0082 - ber_metric: 0.0029 - acc: 0.5352
Epoch 21368/65536
1/1 - 0s - loss: 0.0036 - ber_metric: 9.7656e-04 - acc: 0.5352
Epoch 21369/65536
1/1 - 0s - loss: 0.0172 - ber_metric: 0.0039 - acc: 0.5273
Epoch 21370/65536
1/1 - 0s - loss: 0.0045 - ber_metric: 0.0024 - acc: 0.4961
Epoch 21371/65536
1/1 - 0s - loss: 0.0088 - ber_metric: 0.0034 - acc: 0.5039
Epoch 21372/65536
1/1 - 0s - loss: 0.0073 - ber_metric: 0.0029 - acc: 0.4961
Epoch 21373/65536
1/1 - 0s - loss: 0.0033 - ber_metric: 9.7656e-04 - acc: 0.4883
Epoch 21374/65536
1/1 - 0s - loss: 0.0065 - ber_metric: 0.00

Epoch 21468/65536
1/1 - 0s - loss: 0.0052 - ber_metric: 0.0020 - acc: 0.4688
Epoch 21469/65536
1/1 - 0s - loss: 0.0097 - ber_metric: 0.0029 - acc: 0.4922
Epoch 21470/65536
1/1 - 0s - loss: 0.0086 - ber_metric: 0.0034 - acc: 0.5195
Epoch 21471/65536
1/1 - 0s - loss: 0.0066 - ber_metric: 0.0020 - acc: 0.5078
Epoch 21472/65536
1/1 - 0s - loss: 0.0074 - ber_metric: 0.0020 - acc: 0.5508
Epoch 21473/65536
1/1 - 0s - loss: 0.0072 - ber_metric: 0.0029 - acc: 0.5234
Epoch 21474/65536
1/1 - 0s - loss: 0.0055 - ber_metric: 0.0020 - acc: 0.5117
Epoch 21475/65536
1/1 - 0s - loss: 0.0061 - ber_metric: 0.0015 - acc: 0.5117
Epoch 21476/65536
1/1 - 0s - loss: 0.0022 - ber_metric: 0.0000e+00 - acc: 0.5078
Epoch 21477/65536
1/1 - 0s - loss: 0.0093 - ber_metric: 0.0039 - acc: 0.4922
Epoch 21478/65536
1/1 - 0s - loss: 0.0047 - ber_metric: 0.0020 - acc: 0.5078
Epoch 21479/65536
1/1 - 0s - loss: 0.0084 - ber_metric: 0.0029 - acc: 0.5195
Epoch 21480/65536
1/1 - 0s - loss: 0.0126 - ber_metric: 0.0044 - acc: 0.

Epoch 21574/65536
1/1 - 0s - loss: 0.0036 - ber_metric: 9.7656e-04 - acc: 0.5156
Epoch 21575/65536
1/1 - 0s - loss: 0.0079 - ber_metric: 0.0029 - acc: 0.4727
Epoch 21576/65536
1/1 - 0s - loss: 0.0052 - ber_metric: 0.0015 - acc: 0.4727
Epoch 21577/65536
1/1 - 0s - loss: 0.0060 - ber_metric: 0.0015 - acc: 0.5117
Epoch 21578/65536
1/1 - 0s - loss: 0.0063 - ber_metric: 0.0024 - acc: 0.5469
Epoch 21579/65536
1/1 - 0s - loss: 0.0076 - ber_metric: 0.0024 - acc: 0.4531
Epoch 21580/65536
1/1 - 0s - loss: 0.0063 - ber_metric: 0.0029 - acc: 0.4766
Epoch 21581/65536
1/1 - 0s - loss: 0.0032 - ber_metric: 9.7656e-04 - acc: 0.4883
Epoch 21582/65536
1/1 - 0s - loss: 0.0097 - ber_metric: 0.0034 - acc: 0.5234
Epoch 21583/65536
1/1 - 0s - loss: 0.0089 - ber_metric: 0.0029 - acc: 0.5352
Epoch 21584/65536
1/1 - 0s - loss: 0.0088 - ber_metric: 0.0034 - acc: 0.5352
Epoch 21585/65536
1/1 - 0s - loss: 0.0068 - ber_metric: 0.0015 - acc: 0.4492
Epoch 21586/65536
1/1 - 0s - loss: 0.0083 - ber_metric: 0.0029 - acc

Epoch 21680/65536
1/1 - 0s - loss: 0.0107 - ber_metric: 0.0029 - acc: 0.5195
Epoch 21681/65536
1/1 - 0s - loss: 0.0091 - ber_metric: 0.0024 - acc: 0.5039
Epoch 21682/65536
1/1 - 0s - loss: 0.0046 - ber_metric: 0.0020 - acc: 0.4883
Epoch 21683/65536
1/1 - 0s - loss: 0.0034 - ber_metric: 9.7656e-04 - acc: 0.5352
Epoch 21684/65536
1/1 - 0s - loss: 0.0049 - ber_metric: 0.0020 - acc: 0.4922
Epoch 21685/65536
1/1 - 0s - loss: 0.0081 - ber_metric: 0.0039 - acc: 0.4922
Epoch 21686/65536
1/1 - 0s - loss: 0.0110 - ber_metric: 0.0034 - acc: 0.4883
Epoch 21687/65536
1/1 - 0s - loss: 0.0056 - ber_metric: 9.7656e-04 - acc: 0.5156
Epoch 21688/65536
1/1 - 0s - loss: 0.0043 - ber_metric: 0.0020 - acc: 0.4805
Epoch 21689/65536
1/1 - 0s - loss: 0.0044 - ber_metric: 0.0015 - acc: 0.5352
Epoch 21690/65536
1/1 - 0s - loss: 0.0082 - ber_metric: 0.0029 - acc: 0.5195
Epoch 21691/65536
1/1 - 0s - loss: 0.0056 - ber_metric: 0.0029 - acc: 0.5312
Epoch 21692/65536
1/1 - 0s - loss: 0.0074 - ber_metric: 0.0024 - acc

Epoch 21786/65536
1/1 - 0s - loss: 0.0061 - ber_metric: 0.0024 - acc: 0.4883
Epoch 21787/65536
1/1 - 0s - loss: 0.0040 - ber_metric: 0.0015 - acc: 0.5156
Epoch 21788/65536
1/1 - 0s - loss: 0.0057 - ber_metric: 9.7656e-04 - acc: 0.4570
Epoch 21789/65536
1/1 - 0s - loss: 0.0039 - ber_metric: 0.0020 - acc: 0.5195
Epoch 21790/65536
1/1 - 0s - loss: 0.0070 - ber_metric: 0.0034 - acc: 0.5117
Epoch 21791/65536
1/1 - 0s - loss: 0.0078 - ber_metric: 0.0029 - acc: 0.4688
Epoch 21792/65536
1/1 - 0s - loss: 0.0072 - ber_metric: 0.0024 - acc: 0.4414
Epoch 21793/65536
1/1 - 0s - loss: 0.0083 - ber_metric: 0.0034 - acc: 0.5156
Epoch 21794/65536
1/1 - 0s - loss: 0.0118 - ber_metric: 0.0029 - acc: 0.5000
Epoch 21795/65536
1/1 - 0s - loss: 0.0049 - ber_metric: 0.0015 - acc: 0.4766
Epoch 21796/65536
1/1 - 0s - loss: 0.0131 - ber_metric: 0.0034 - acc: 0.4609
Epoch 21797/65536
1/1 - 0s - loss: 0.0080 - ber_metric: 0.0020 - acc: 0.4727
Epoch 21798/65536
1/1 - 0s - loss: 0.0058 - ber_metric: 0.0015 - acc: 0.

In [ ]:
computeVarianceAWGN(encoderAWGN,do_polar=True,print_code=True,round=False)

In [ ]:
testLinearAWGN(encoderAWGN)

In [ ]:
np.count_non_zero(np.round(encoderAWGN.predict(np.eye(256))))

In [ ]:
print()